In [9]:
import plotly.graph_objects as go
import pandas as pd
import sqlite3

In [10]:
def carregar_dados():
    conn = sqlite3.connect('../data/runningDB.sqlite3')
    df = pd.read_sql_query("SELECT * FROM running_data", conn)
    conn.close()
    
    # tratamento
    df['data'] = pd.to_datetime(df['data'])
    df['quilometragem'] = df['quilometragem'].astype(float)
    
    # Calcular calorias por minuto e km por minuto
    df['calorias_por_minuto'] = df['calorias'] / df['tempo_min']
    df['km_por_minuto'] = df['quilometragem'] / df['tempo_min']
    df['dia_semana'] = df['data'].dt.day_name()
    
    return df

In [11]:
def criar_grafico_barras_verticais(valores_corrente, valores_anterior, titulo):
    fig = go.Figure()

    fig.add_trace(go.Bar(
        x=['Mês Corrente', 'Mês Anterior'],
        y=[valores_corrente, valores_anterior],
        marker=dict(color=['#212b2c', '#536c6f']),
        text=[valores_corrente, valores_anterior],
        textposition='auto'
    ))

    fig.update_layout(
        title=titulo,
        xaxis_title='Mês',
        yaxis_title='Valores',
        plot_bgcolor='lightgrey',
        paper_bgcolor='lightgrey',
        font=dict(color='black'),
        title_font=dict(color='black'),
        xaxis=dict(
            title_font=dict(color='black'),
            tickfont=dict(color='black')
        ),
        yaxis=dict(
            title_font=dict(color='black'),
            tickfont=dict(color='black')
        ),
        legend=dict(font=dict(color='black')),
        autosize=True,
        margin=dict(l=100, r=20, t=50, b=50)
    )

    return fig


In [12]:
df = carregar_dados()

In [15]:
df.head()

,data,quilometragem,tempo_min,calorias,dia_semana,calorias_por_minuto,km_por_minuto
0,2024-06-09,0.0,0,0,Sunday,NaN,NaN
1,2024-06-10,4.5,40,294,Monday,7.350000,0.112500
2,2024-06-11,5.0,45,329,Tuesday,7.311111,0.111111
3,2024-06-12,4.7,40,311,Wednesday,7.775000,0.117500
4,2024-06-13,5.1,45,329,Thursday,7.311111,0.113333


In [16]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 79 entries, 0 to 78
Data columns (total 7 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   data                 79 non-null     datetime64[ns]
 1   quilometragem        79 non-null     float64       
 2   tempo_min            79 non-null     int64         
 3   calorias             79 non-null     int64         
 4   dia_semana           79 non-null     object        
 5   calorias_por_minuto  44 non-null     float64       
 6   km_por_minuto        44 non-null     float64       
dtypes: datetime64[ns](1), float64(3), int64(2), object(1)
memory usage: 4.4+ KB


In [13]:
valores_corrente = df.groupby(df['data'].dt.to_period('M'))['calorias'].sum().iloc[-1]
valores_anterior = df.groupby(df['data'].dt.to_period('M'))['calorias'].sum().iloc[-2]
titulo_grafico = "Média de Calorias por Dia"

In [14]:
criar_grafico_barras_verticais(valores_corrente, valores_anterior, titulo_grafico)